In [43]:
# Setup
from pathlib import Path
import sys
ROOT = Path.cwd().resolve().parents[1]
sys.path.insert(0, str(ROOT))
from gu_toolkit import *

# Plotting with `gu_toolkit`: a comprehensive tour

This notebook walks through the most common plotting workflows you will use with the toolkit.
We focus on *clean math exploration patterns* that work well in teaching, research, and quick exploratory analysis.

By the end you will see how to:

- Build plots from SymPy expressions.
- Layer multiple traces on one figure.
- Add interactive parameters (sliders) automatically or manually.
- Customize ranges, sampling density, and styling.
- Use global `plot(...)` and figure contexts for concise demos.
- Create lightweight *info panels* that respond to parameter changes.
- Prototype common calculus and modeling ideas.

> **Note:** This notebook assumes you are running in Jupyter or JupyterLab so that widgets and Plotly figures render inline.


## 1. Plotting with context managed syntax

`SmartFigure` accepts SymPy expressions directly. The figure auto-compiles them to numerical expressions and renders them.

Add multiple plots to the same figure to compare functions. 

The `plot` method accepts common styling arguments. You can pass:
- `color`,
- `thickness`,
- `dash`
- `opacity`
- full Plotly `line`/`trace` dictionaries for advanced settings.

In [44]:
fig1 = Figure(x_range=(-6, 6), y_range=(-2.5, 2.5))
display(fig1)

with fig1:
    set_title("Sin wave")
    plot(x, sin(x), id="sin")
    plot(x, cos(x), id="cos", dash="dash", color="#d62728")
    plot(x, sin(2 * x), id="sin2", thickness=20, opacity=0.1)
    plot(x, sin(5 * x), id="sin5", dash="dot", color="#d62728", opacity=0.3,)


OneShotOutput()

If you reuse an `id`, the trace is updated instead of replaced.

In [45]:
with fig1:
    plot(x, sin(x**2), id="sin")

## 2. Parameters (sliders)

When an expression contains symbols besides the plot variable, `SmartFigure` automatically creates sliders
for them. The parameter symbols are inferred from the expression.


In [47]:
fig2 = Figure(x_range=(-6, 6), y_range=(-3, 3))
display(fig2)
with fig2:
    set_title("Auto-created parameters")
    plot(x, a * sin(x), id="a_sin")
    plot(x, cos(x + b), id="b_shift")


OneShotOutput()

### Adjusting slider defaults

You can control slider ranges and defaults by calling `parameter` directly. This is especially helpful
when you want a parameter to start at a specific value or use a specific range/step size.


In [48]:
with fig2:
    parameter(a, min=-2, max=2, value=1, step=0.1)
    parameter(b, min=-3.14, max=3.14, value=0.0, step=0.05)

In [49]:
with fig2:
    parameter(a, min=3, max=5, value=4, step=0.1)
    parameter(b, min=-6.14, max=-5.14, value=-6, step=0.05)

### Saving parameter values

In [50]:
with fig2:
    param_values = parameters.snapshot()
param_values

{a: 3.0, b: -5.989999999999999}

In [51]:
with fig2:
    param_values_full = parameters.snapshot(full=True)
param_values_full

ParameterSnapshot({a: {'value': 3.0, 'capabilities': ['default_value', 'min', 'max', 'step'], 'default_value': 0.0, 'min': 3.0, 'max': 5.0, 'step': 0.1}, b: {'value': -5.989999999999999, 'capabilities': ['default_value', 'min', 'max', 'step'], 'default_value': 0.0, 'min': -6.14, 'max': -5.14, 'step': 0.05}})

## 3. Customizing the domain and sampling density

Use `x_domain` (per-trace) and `sampling_points` to refine how curves are sampled. This is especially
useful for rapidly oscillating functions.


In [52]:
fig3 = Figure(x_range=(-1, 1), y_range=(-2, 2), sampling_points=200)
display(fig3)
with fig3:
    set_title("Sampling and domain control")
    plot(x, sin(15 * x), id="dense", color="#1f77b4")
    plot(
        x,
        sin(15 * x),
        id="dense_zoomed",
        x_domain=(-0.5, 0.5),
        sampling_points=10,
        dash="dot",
        color="#ff7f0e",
    )

OneShotOutput()

## 4. Exploring plotted expressions

In [ ]:
fig4 = Figure(x_range=(-6, 6), y_range=(-2.5, 2.5))
display(fig4)

with fig4:
    set_title("Sin wave")
    plot(x, sin(x), id="f1")
    plot(x, cos(x), id="f2")
    plot(x, a + sin(2 * b * x), id="f3")

### Symbolic vs. numpified expressions

Each entry in `plots` is a `SmartPlot`. Useful views are:

- `plot.symbolic_expression`: the original SymPy expression.
- `plot.numpified`: the compiled callable with explicit positional arguments.
- `plot.numeric_expression`: a convenience wrapper that is the same as `numpified` but automatically reads current slider values.

This split is handy when you want fast numeric evaluation outside rendering, or when you want to freeze parameters and compare scenarios.


In [ ]:
with fig4:
    f1 = plots["f1"]
    f3 = plots["f3"]

display(f1.symbolic_expression)
display(f3.symbolic_expression)
display(f3.numpified)
display(f3.numeric_expression)

### Evaluating the numpified function directly

`f3.numpified` expects arguments in the order shown by `f3.numpified.args`, i.e. `(x, a, b)` for this plot.


In [ ]:
f3 = fig4.plots["f3"]
f3.numpified.args

In [ ]:
x_values = np.linspace(-2, 2, 5)
y_values = f3.numpified(x_values, 0.5, 1.5)
display(f3.numeric_expression.args)
display(x_values)
display(y_values)
display("---")
y_values = f3.numpified(1, 0.5, 1.5)
display(f3.numeric_expression.args)
display(f3.numeric_expression(1))

### Snapshot parameters, bind, and compare

Use `parameters.snapshot()` to capture current slider values. Binding to that snapshot creates a **dead/frozen** callable that does not change when sliders move.


In [ ]:
with fig4:
    parameter(a, min=-2, max=2, value=0.25, step=0.05)
    parameter(b, min=0.5, max=3.0, value=1.0, step=0.05)

x_values = np.linspace(-2, 2, 5)
snapshot = fig4.parameters.snapshot()
frozen = f3.numeric_expression.bind(snapshot)

with fig4:
    parameters[a].value = 1.2
    parameters[b].value = 2.4

y_frozen = frozen(x_values)
y_live = f3.numeric_expression(x_values)

display(snapshot)
display(y_frozen)
display(y_live)

### Context-managed `.bind()` for live parameter access

`NumpifiedFunction.bind()` can read from the **active figure context** when called with no argument.

- Inside `with fig4:`, `f3.numpified.bind()` returns a live callable bound to `fig4` parameters.
- Outside a figure context, call `f3.numpified.bind(fig4)` explicitly.


In [ ]:
with fig4:
    live_ctx = f3.numpified.bind()
    y_ctx_1 = live_ctx(x_values)
    parameters[a].value = -0.75
    parameters[b].value = 0.8
    y_ctx_2 = live_ctx(x_values)

live_explicit = f3.numpified.bind(fig4)
y_explicit = live_explicit(x_values)

display(y_ctx_1)
display(y_ctx_2)
display(y_explicit)
display(live_ctx.unbind())

## 5. Numeric integration with `NIntegrate`

`NIntegrate` accepts symbolic expressions, numpified functions, bound numpified functions, and plain numeric callables.
It also supports `binding=` for supplying parameter values from dictionaries or `SmartFigure` objects.


In [ ]:
# SymPy expression with explicit binding map
NIntegrate(a * x + b, (x, 0, 1), binding={a: 2.0, b: 3.0})

In [ ]:
fig5 = SmartFigure()
fig5.parameter([a, b], value=0)
fig5.parameters[a].value = 2.0
fig5.parameters[b].value = 3.0

# Use figure binding explicitly
NIntegrate(a * x + b, (x, 0, 1), binding=fig5)

In [ ]:
# With an active figure context, binding can be omitted
with fig5:
    NIntegrate(a * x + b, (x, 0, 1))

In [ ]:
# Plain callable: the symbol in (x, a, b) is ignored for numeric callables
NIntegrate(lambda t: t**2, ("ignored_symbol", 0, 1))

In [ ]:
# Unbound callable with parameters resolved via binding
def linear(t, a, b):
    return a * t + b


NIntegrate(linear, (x, 0, 1), binding={a: 2.0, b: 3.0})

In [ ]:
# SymPy Lambda (function form) with explicit parameter binding
lam = sp.Lambda((x, a, b), a * x + b)
NIntegrate(lam, (x, 0, 1), binding={a: 2.0, b: 3.0})

In [ ]:
# SymPy Lambda can also use SmartFigure binding or active context
NIntegrate(lam, (x, 0, 1), binding=fig5)
with fig5:
    NIntegrate(lam, (x, 0, 1))

## 6. Real Fourier series with `NReal_Fourier_Series`

`NReal_Fourier_Series(expr, (x,a,b), samples=...)` returns `(cos_coeffs, sin_coeffs)`
for the L2-normalized real trigonometric basis on `(a,b)` with frequencies
`2*pi*n/(b-a)`.


In [ ]:
cos_coeffs, sin_coeffs = NReal_Fourier_Series(
    sin(3 * x) + 0.5 * cos(2 * x), (x, 0, 2 * pi), samples=4096
)
cos_coeffs[:8], sin_coeffs[:8]

In [ ]:
# With parameterized expressions and figure-driven binding
fig6 = Figure()
fig6.parameter(k, value=3.0)
expr_k = sin(k * x)
NReal_Fourier_Series(expr_k, (x, 0, 2 * pi), samples=4096, binding=fig6)[1][:8]

## 6. Audio playback with `play`

`play(expr, (x, a, b), loop=True)` converts a 1D SymPy expression to audio in JupyterLab.

- `expr` is numpified internally and evaluated over the time interval `[a, b)` in **seconds**.
- `x` is the expression's time symbol.
- The signal is normalized to avoid clipping and emitted as a WAV audio widget.
- `loop=True` restarts playback automatically when the interval ends.

This is useful for quick sonification of symbolic expressions while iterating in notebooks.


In [ ]:
# A 2-second tone sweep from 220 Hz to 660 Hz
sweep = sin(2 * pi * (220 * x + 110 * x**2))

play(sweep, (x, 0, 2), loop=False)

### Looping playback

Set `loop=True` to keep replaying the generated sound until paused from the audio controls.


In [ ]:
# Loop a simple harmonic blend over one second
loop_expr = 0.6 * sin(2 * pi * 440 * x) + 0.3 * sin(2 * pi * 660 * x)
play(loop_expr, (x, 0, 1), loop=True)

### Parameterized expressions

`play` expects a single-variable expression in the provided symbol.
If your expression contains additional symbols (for example `a`), substitute or bind them first:

```python
play((a*sin(2*pi*440*x)).subs({a: 0.8}), (x, 0, 1))
```

Tip: keep amplitudes moderate (roughly within `[-1, 1]`) for predictable loudness.
